<a href="https://colab.research.google.com/github/sandhrabijoy/Store-Item-Demand-Forecasting/blob/main/SalesForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setting up basics for forecasting pipeline


In [4]:
import numpy as np
import pandas as pd

import math
import itertools

#matplotlib libraries
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors
import seaborn as sns

#date libraries
from dateutil import parser
from datetime import datetime, timedelta, date
import holidays

#prophet library
from prophet import Prophet
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric
from prophet.diagnostics import cross_validation

#pandas options
pd.set_option('display.float_format', lambda x:'%.2f' % x)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows', None)

#matplotlib setting defaults
sns.set(font='Arial',rc={
    "axes.axisbelow":False,
    "axes.edgecolor":"lightgrey",
    "axes.facecolor":"None",
    "axes.grid":False,
    "axes.labelcolor":"dimgrey",
    "axes.spines.right":False,
    "axes.spines.top":False,
    "figure.facecolor":"white",
    "lines.solid_capstyle":"round",
    "patch.edgecolor":"w",
    "patch.force_edgecolor":True,
    "text.color":"dimgrey",
    "xtick.bottom":False,
    "xtick.color":"dimgrey",
    "xtick.direction":"out",
    "xtick.top":False,
    "ytick.color":"dimgrey",
    "ytick.direction":"out",
    "ytick.left":False,
    "ytick.right":False
})

In [5]:
def missing_data(input_data):
  total = input_data.isnull().sum()
  percent = (input_data.isnull().sum()/input_data.isnull().count()*100)
  table = pd.concat([total,percent],axis=1,keys=['Total','Percent'])
  types=[]
  for col in input_data.columns:
    dtype=str(input_data[col].dtype)
    types.append(dtype)
  table["Types"]=types
  return(pd.DataFrame(table))

def mape(actual,pred):
  actual,pred=np.array(actual),np.array(pred)
  return np.mean(np.abs(actual-pred)/actual)*100